In [ ]:
import folium as fm
import pandas as pd
import param
import panel as pn
import random
pn.extension(sizing_mode="stretch_width")
import geopandas as gpd

# example

# our project

In [ ]:
def get_map(lat=41.89051776994878, long=-87.62846577303873, zoom_start=11):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()

pn.panel(map, height=800)

In [ ]:
# load the data
chicago_data = gpd.read_file('data/chicago_data.geojson').set_crs("epsg:3857", allow_override=True)
bus_ridership = gpd.read_file('data/bus_ridership.geojson')
#bus_ridership.to_crs("epsg:3857", inplace = True)

In [ ]:
bus_ridership

In [ ]:
#### then used Folium - Folium is better (this chunk of code is not on Panel)
import folium
from folium.plugins import MarkerCluster
## choropleth for boardings (will be predicted demand for our project)
def choose_x(x='boardings'):    
    # Minimum
    min_val = chicago_data[x].min()

    # Maximum
    max_val = chicago_data[x].max()

    # Calculate a normalized column
    normalized = (chicago_data[x] - min_val) / (max_val - min_val)

    # Add to the dataframe
    chicago_data["normalized"] = normalized
    
choose_x()
def add_choropleth(map, chicago_data, x='boardings'):
    import matplotlib.colors as mcolors
    from matplotlib import pyplot as plt
    # Use a red-purple colorbrewer color scheme
    cmap = plt.get_cmap('RdPu')


    def get_style(feature):
        # Get the data value from the feature
        value = feature['properties']['normalized']

        # Evaluate the color map
        # NOTE: value must between 0 and 1
        rgb_color = cmap(value) # this is an RGB tuple

        # Convert to hex string
        color = mcolors.rgb2hex(rgb_color)

        # Return the style dictionary
        return {'weight': 0.5, 'color': color, 'fillColor': color, "fillOpacity": 0.75}

    def get_highlighted_style(feature):
        return {"weight": 3, "color": "black"}


    # Add the GeoJson to the map
    folium.GeoJson(
        chicago_data.to_crs(epsg=4326).to_json(), # IMPORTANT: make sure CRS is lat/lng (EPSG=4326)
        #name='Philadelphia ZIP_codes',
        style_function=get_style,
        #highlight_function=get_highlighted_style,
        tooltip=folium.GeoJsonTooltip([x])
    ).add_to(map)
    
#def add_marker_cluster(map, bus_ridership):
    TOR_COORDINATES = (pd.to_numeric(bus_ridership['latitude']).mean(), pd.to_numeric(bus_ridership['longitude']).mean())

    # subset to match subset of locations
    MAX_RECORDS = 2500

    mc = MarkerCluster()
##possible reason 
    # iterate through dataset to create clusters
    for row in bus_ridership[0:MAX_RECORDS].itertuples():
        mc.add_child(folium.Marker(location=[row.latitude,  row.longitude],
                     popup=row.boardings))

    map.add_child(mc)
    
add_choropleth(map, chicago_data)
#add_marker_cluster(map, bus_ridership)
pn.panel(map, height=800)

In [ ]:
columns = list(chicago_data.columns[2:-1])
class PanelFoliumMap(param.Parameterized):
    x = param.Selector(default='boardings', objects=columns)
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=800, margin=0)    
        self.view = pn.Column(
            self.param.x,
            self.folium_pane,
            sizing_mode="stretch_both", height=800
        )
        self._update_map()
        
    @param.depends("x", watch=True)
    def _update_map(self):
        self.map = get_map()
        choose_x(self.x)
        add_choropleth(self.map, chicago_data, self.x)
        self.folium_pane.object = self.map

        
app = PanelFoliumMap()
app.view

In [ ]:
# The title
title = pn.Pane("<h1>Bus Ridership in Chicago by Block</h1>", width=1000)
# Layout the dashboard
panel = pn.Column(
    pn.Row(title),
    pn.Row(app.view),
)
panel.servable()